In [ ]:
import hashlib
import random
from  pyprimes import isprime

class DSA():
    def _pow(self, a, e, m):
        i == 1
        result = a % m
        while i <= e:
            result = ( result * result ) % m
            i += i
        return result

    def subGroupGenerator(self, p, q):
        e = (p-1) // q
        g = 1
        while g == 1:
            h = random.randint(0, p-1)
            g = _pow(h,e,p)
        return g
        
    def egcd(self, a, b):
        x,y, u,v = 0,1, 1,0
        while a != 0:
            q, r = b//a, b%a
            m, n = x-u*q, y-v*q
            b,a, x,y, u,v = a,r, u,v, m,n
        gcd = b
        return gcd, x, y

    def modinv(self, a, m):
        gcd, x, y = egcd(a, m)
        if gcd != 1:
            return None  # modular inverse does not exist
        else:
            return x % m

    #Post-condition: returns public parameters q,p,g
    def DL_Param_Generator(self, small_bound, large_bound):
        L = small_bound
        N = large_bound
        seedlen = N 
        q = 4
        U = 2
        domain_parameter_seed = 0
        while not isprime(q):
            # domain_parameter_seed = number.getPrime(seedlen)
            domain_parameter_seed =int(random.getrandbits(seedlen))
            U = hash(domain_parameter_seed % pow(2, N-1))
            q = pow(2, N-1) + U + 1 - (U%2)

        # print("q:", q)

        offset = 1 
        V = list()
        for counter in range(0, 4*L-1):
            outlen = (U.bit_length())
            n = L // outlen -1
            b = L - 1 - (n*outlen)
            for j in range(0, n):
                V.append(hash((domain_parameter_seed+offset+j)%pow(2, seedlen)))
            W = 0 
            for i in range(0,n-1):
                W += V[i] * pow(2, (i) * outlen)
            W += (V[-1] % pow(2,b)) * pow(2, n * outlen)
            X = W + pow(2, L-1)
            c = X % (2*q)
            p = X - (c-1)
            if isprime(p):
                g = subGroupGenerator(p, q)
                return q,p,g
            offset += n + 1
        return (0,0,0)



    #Pre-condition: it takes q, p, g as parameter
    #Prodecure: A user picks a random secret key 0 < α < q and computes the public key β = gα mod p.
    #Post-condition: returns key tuple as (alpha, beta)
    def KeyGen(self, p, q, g):
        alpha = randint(1,q)
        beta = pow(g,alpha,p)
        return alpha, beta

    # Pre-condition: takes message, secret key(alfa), q, p and genrator(g)
    # Post-condition: returns signature tuple as (r,s)
    def SignGen(self, m, p, q, g, alpha, beta):
        h = hashlib.sha3_256(m).hexdigest()
        h = h % q
        k = randint(1,q)
        r = pow(g,k,p)
        s = (alpha*r +k*h) % q

        return r, s

    #Pre-condition: takes message, signature tuple(r,s), q, p, g and public key(beta)
    #Post-condtion: if condition holds it returns verification true, otherwise false
    def SignVer(self,m, r, s, p, q, g, beta):
        h = hashlib.sha3_256(m).hexdigest()
        h = h % q
        v = modinv(h) % q
        z1 = (s*v) % q
        z2 = ((q-r)*v) % q
        u = (pow(g,z1,p) + pow(beta,z2,p)) % p
        if r == (u%q):
            return True
        else:
            return False




In [ ]:
import string
import DSA
import random

class TxGen:
    
    def hash_me(self, transaction):
        transaction = transaction.encode()
        h = hashlib.sha3_256(transaction).hexdigest()
        return h
    
    
    def id_generator(self,size=10, chars=string.ascii_uppercase + string.digits):
        return ''.join(random.choice(chars) for _ in range(size))
    def GenSingleTx(p, q, g, alpha, beta):
        transaction = ""
        transaction.join("*** Bitcoin transaction ***\n")

        line = "Serial number: "
        x = uuid.uuid4().int
        line += str(x) + "\n"
        transaction.join(line)
        
        line = "Payer: " + id_generator() + "\n"
        transaction.join(line)

        line = "Payee: " + id_generator() + "\n"
        transaction.join(line)

        line = "Amount: "
        y = randint(1,1000)
        line += str(y) + " Satoshi\n"
        transaction.join(line)
        
        small_bound = 1 << 256 
        large_bound = 1 << 2048
        q, p, g = DSA.DL_Param_Generator(small_bound, large_bound)
        
        line = "p: " + str(p) + "\n"
        transaction.join(line)
        
        line = "q: " + str(q) + "\n"
        transaction.join(line)
        
        line = "g: " + str(g) + "\n"
        transaction.join(line)
        
        alpha, beta = DSA.KeyGen(p, q, g)
        
        line = "Public Key (beta): " + str(beta) + "\n"
        transaction.join(line)
        
        m = hash_me(transaction)
        
        (r, s) = DSA.SignGen(m, p, q, g, alpha, beta)
        
        line = "Signature (r): " + str(r) + "\n"
        transaction.join(line)
        
        line = "Signature (s): " + str(s) + "\n"
        transaction.join(line)
        
        return transaction


